In [1]:
!pip install transformers datasets peft accelerate --quiet

In [47]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [48]:
import pandas as pd

# Load CSV
df = pd.read_csv('calender1.csv')

# Combine relevant fields into flat records
# (e.g., for each row: "Date: ..., Time: ..., Event: ..., Location: ...")
raw_texts = df.astype(str).apply(lambda row: " | ".join(row.values), axis=1).tolist()

In [49]:
from datasets import Dataset

# Convert raw text into HuggingFace Dataset
text_dataset = Dataset.from_dict({"text": raw_texts})

# Shuffle and split for training/evaluation
dataset = text_dataset.train_test_split(test_size=0.1)

In [50]:
from transformers import GPT2TokenizerFast
from datasets import Dataset

# STEP 1: Load tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# FIX: Set pad_token = eos_token
tokenizer.pad_token = tokenizer.eos_token

# STEP 2: Prepare raw text
raw_texts = df.astype(str).apply(lambda row: " | ".join(row.values), axis=1).tolist()
dataset = Dataset.from_dict({"text": raw_texts})

def tokenize_function(batch):
    tokens = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()  # This is required for causal LM
    return tokens

C:\Users\hvard\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [51]:
# STEP 4: Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
# Split tokenized dataset for training and evaluation
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [52]:
from transformers import AutoModelForCausalLM

# Load OpenELM model from Apple with remote code enabled
model = AutoModelForCausalLM.from_pretrained("apple/OpenELM-270M", trust_remote_code=True)

# Match tokenizer pad token ID with model config
model.config.pad_token_id = tokenizer.pad_token_id

In [53]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./openelm-calendar-finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    fp16=False,             # Disable mixed precision (GPU-only)
    no_cuda=True            # <-- Force CPU
)

C:\Users\hvard\anaconda3\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\hvard\anaconda3\Lib\site-packages\transformers\training_args.py:1540: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [56]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    tokenizer=tokenizer
)

trainer.train()

***** Running training *****
  Num examples = 14
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total optimization steps = 60

Epoch 1/3
100%|████████████████████████████████████████| 15/15 [00:12<00:00,  1.24it/s]
Step 20: loss = 1.20
Saving model checkpoint to ./openelm-calendar-finetuned/checkpoint-20
Configuration saved in ./openelm-calendar-finetuned/checkpoint-{epoch * 20}/config.json
Model weights saved in ./openelm-calendar-finetuned/checkpoint-{epoch * 20}/pytorch_model.bin
tokenizer config file saved in ./openelm-calendar-finetuned/checkpoint-{epoch * 20}/tokenizer_config.json
Special tokens file saved in ./openelm-calendar-finetuned/checkpoint-{epoch * 20}/special_tokens_map.json

Epoch 2/3
100%|████████████████████████████████████████| 15/15 [00:12<00:00,  1.24it/s]
Step 40: loss = 1.10
Saving model checkpoint to ./openelm-calendar-finetuned/checkpoint-40
Configuration saved in ./openelm-calendar-finetuned/checkpoint-{epoch * 20}/config.json
Model weights save

In [57]:
# Save the final model and tokenizer
model.save_pretrained("./openelm-calendar-finetuned")
tokenizer.save_pretrained("./openelm-calendar-finetuned")

print("✅ Model and tokenizer saved to './openelm-calendar-finetuned'")

✅ Model and tokenizer saved to './openelm-calendar-finetuned'
